In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

DATA_DIR = '/content/drive/MyDrive/datathon_2022/data'

In [ ]:
ls

Potential questions:

- What factors are impacting loans? 
- What are different types of customers?

K-Mode clustering for mixed numeric snd categorical variables: 
https://towardsdatascience.com/clustering-algorithm-for-data-with-mixed-categorical-and-numerical-features-d4e3a48066a0

Python factor analysis library (PCA, CA, MCA, MFA, FAMD):
https://github.com/MaxHalford/prince#factor-analysis-of-mixed-data-famd


In [ ]:
df = pd.read_pickle('accepted_merged.pickle')

In [ ]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
df['m_since_earliest_cr'] = ((df['issue_d'] - df['earliest_cr_line']) / np.timedelta64(1, 'M')).astype(int)

In [ ]:
col_to_drop = ['earliest_cr_line', 'application_type', 'id']
df = df.drop(columns=col_to_drop)

In [ ]:
loan_cols = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 
             'int_rate', 'installment', 'grade', 'sub_grade', 'issue_d', 
             'issue_d_year', 'loan_status']
X = df.drop(columns=loan_cols)

## Lender Clustering

In [ ]:
# !pip install sweetviz

In [ ]:
# import sweetviz as sv

# my_report = sv.analyze(X, pairwise_analysis='off')
# my_report.show_html(filepath=os.path.join(DATA_DIR, 'SWEETVIZ_REPORT.html'))

In [ ]:
!pip install kmodes

In [ ]:
X.head()

In [ ]:
X['emp_length'] = X['emp_length'].replace({"< 1 year": "0 year"})
X['emp_length'] = X['emp_length'].apply(lambda x: str(x[:2].strip()))
emp_categories = np.array(['UNK'] + [str(i) for i in range(0,11)])
X['emp_length'] = pd.Categorical(X['emp_length'], categories=emp_categories, ordered=True).fillna('UNK')

X['dti'] = X['dti'].fillna(X['dti'].max())
X['dti_cat'] = X['dti_cat'].fillna('100-1000')

In [ ]:
numerical_cols = [i for i in X.columns if X[i].dtype == float]
X[numerical_cols] = X[numerical_cols].fillna(0)

In [ ]:
from kmodes.kprototypes import KPrototypes
kproto = KPrototypes(n_clusters=2, verbose=2, max_iter=20)
kproto.fit(X, categorical=[0,1,3,4,5,6,87,88,89])

## FAMD for column factors

In [ ]:
# pip install prince

In [ ]:
import prince
famd = prince.FAMD(
    n_components=3,
    n_iter=3,
    check_input=True,
    engine='auto',
    random_state=42)
famd = famd.fit(X)